### Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gpytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
import math

### Read Data

In [ ]:
all_data_path = "./data"
output_path = "./consolidated_data"
csv_paths = []
data_file_dict = {}
idx = 0
for f in os.walk(all_data_path):
    rootFolder = f[0]
    for fname in f[-1]:
        if (".csv" in fname):
            csv_paths.append(os.path.join(rootFolder, fname))
            data_file_dict[fname] = idx
            idx += 1
df = pd.read_csv(csv_paths[data_file_dict['ColdHardiness_Grape_Merlot.csv']])

### defining features and labels

In [ ]:
timeseries_length = 30 # days
# exclude SEASON_JDAY
features = [#'SEASON_JDAY',
       'PREDICTED_Hc', 'MEAN_AT', 'MIN_AT', 'AVG_AT', 'MAX_AT',
       'MIN_REL_HUMIDITY', 'AVG_REL_HUMIDITY', 'MAX_REL_HUMIDITY', 'MIN_DEWPT',
       'AVG_DEWPT', 'MAX_DEWPT', 'P_INCHES', 'WS_MPH', 'MAX_WS_MPH',
       'LW_UNITY', 'ETO',
       'ETR']
# PREDICTED_Hc, MEAN_AT, MIN_AT, MAX_AT, MIN_REL_HUMIDITY, AVG_REL_HUMIDITY, MAX_REL_HUMIDITY, MIN_DEWPT, AVG_DEWPT, MAX_DEWPT, P_INCHES,WS_MPH, MAX_WS_MPH
# label = ['LT10', 'LT50', 'LT90']
label = ['LT50']
print(len(features))
# # get all indexes where LT10 is not null
# idx_LT10_not_null = df[df['LT10'].notnull()].index.tolist()
idx_LT50_not_null = df[df['LT50'].notnull()].index.tolist()
# idx_LT90_not_null = df[df['LT90'].notnull()].index.tolist()

# idx_LT_not_null = set(idx_LT10_not_null) & set(idx_LT50_not_null) & set(idx_LT90_not_null) # intersection where LT10, LT50, LT90 are not null
idx_LT_not_null = idx_LT50_not_null
print(len(idx_LT_not_null))
dormant_seasons = df[df["DORMANT_SEASON"] == 1]
display(dormant_seasons)

### Data Preprocessing

In [ ]:
# create array of timeseries examples
# timeseries_idx = []

# for _, idx in enumerate(idx_LT_not_null):
#     if (idx - timeseries_length) > 0:
#         timeseries_idx.append(np.arange(idx - timeseries_length + 1, idx + 1))

# timeseries_idx = np.array(timeseries_idx)

# print(idx_LT_not_null)
# print("---")
# print(timeseries_idx.shape)
# print(timeseries_idx[:5])
# # display(df.loc[timeseries_idx[0].tolist()])

seasons = []
last_x = 0
idx = -1
season_max_length = 0
for x in df[df["DORMANT_SEASON"] == 1].index.tolist():
    if x - last_x > 1:
        seasons.append([])
        if idx > -1:
            season_max_length = max(season_max_length, len(seasons[idx]))
        idx += 1
    seasons[idx].append(x)
    last_x = x
season_max_length = max(season_max_length, len(seasons[idx]))
print(len(seasons))
print([len(x) for x in seasons])
print(season_max_length)
print("max:")
print(df[features].max())
print("min:")
print(df[features].min())
print("median:")
print(df[features].median())

print("number of missing temperatures")
print((df == -100).sum())
df[features].hist(figsize = (60, 50))

### Linear Interpolation of missing values

In [ ]:
def linear_impute(number, last, next):
    return [ last + (next - last) / (number + 1) * (n + 1) for n in range(number)]
def data_prepocess(x, is_one_hot = True, is_normal = True):
    # 4. YEAR_JDAY needs to be one-hot encoded
    def one_hot_encoded(data, idx, one_hot_size, divided_by):
        data = np.array(data)
        data = np.nan_to_num(data)
        batch, r_size, f_size = data.shape
        return_data = np.zeros((batch, r_size, f_size + one_hot_size - 1))
        for i, _x in enumerate(data):

            oh_feature = _x[:, idx]

            one_hot_v = np.zeros((oh_feature.shape[idx], one_hot_size))
            one_hot_v[np.arange(oh_feature.size), np.floor(oh_feature / divided_by).astype(int)] = 1
            return_data[i] = np.concatenate((_x[:, 1:], one_hot_v), axis = 1)

        return return_data
        # idx of YEAR_JDAY is 0
    if is_one_hot:
        x_m = one_hot_encoded(x, 0, 12, 31)
    else:
        x_m = np.nan_to_num(np.array(x))
        
    # 5. Data needs to be normalized.
    if not is_one_hot:
        NORMALIE_FEATURES = features
    else:
        NORMALIE_FEATURES = [#'SEASON_JDAY',
       'PREDICTED_Hc', 'MEAN_AT', 'MIN_AT', 'MAX_AT',
       'MIN_REL_HUMIDITY', 'AVG_REL_HUMIDITY', 'MAX_REL_HUMIDITY', 'MIN_DEWPT',
       'AVG_DEWPT', 'MAX_DEWPT', 'P_INCHES', 'WS_MPH', 'MAX_WS_MPH']#, "diff_Hc"]
    NORMALIE_FEATURES_idx = np.array(range(0, len(NORMALIE_FEATURES)))
    if is_normal:
        x_mean = df[NORMALIE_FEATURES].mean().to_numpy()
        x_std = df[NORMALIE_FEATURES].std().to_numpy()
        x_min = df[NORMALIE_FEATURES].min().to_numpy()
        x_max = df[NORMALIE_FEATURES].max().to_numpy()
        print(x_mean)
        print(x_std)
        # print(x_m[0, 0:4, :])
        x_m[:, :, NORMALIE_FEATURES_idx] = (x_m[:, :, NORMALIE_FEATURES_idx] - x_mean) / x_std
        # x_m[:, :, NORMALIE_FEATURES_idx] = (x_m[:, :, NORMALIE_FEATURES_idx] - x_min) / (x_max - x_min)
        # print(x_m[0, 0:4, :])
        # print(x_m.shape)
    
    return x_m

last_value = None
miss_flag = False
miss_idx = []
lable_with_miss_value = df.loc[: , (df == -100).any()].columns
for lb in lable_with_miss_value:
    for i, value in enumerate(df[lb]):
        if value == -100:
            if not miss_flag:
                last_value = df[lb][i - 1]
                miss_flag = True
            miss_idx.append(i)
        else:
            if miss_flag:
                fill_values = linear_impute(len(miss_idx), last_value, value)
                for j, mi in enumerate(miss_idx):
                    df.loc[mi,lb] = fill_values[j]
                miss_flag = False
                miss_idx = []
                last_value = None

### Data processing

In [ ]:
# 3. imputate the missing Predicted_Hc  

lb = "PREDICTED_Hc"
miss_flag = False
start_flag = False
miss_idx = []
for i, value in enumerate(df[lb]):
    if pd.isnull(value) and not start_flag:
        continue
    else:
        start_flag = True
    if pd.isnull(value):
        if not miss_flag:
            last_value = df[lb][i - 1]
            miss_flag = True
        miss_idx.append(i)
    else:
        if miss_flag:
            fill_values = linear_impute(len(miss_idx), last_value, value)
            for j, mi in enumerate(miss_idx):
                df.loc[mi,lb] = fill_values[j]
            miss_flag = False
            miss_idx = []
            last_value = None
            
# other data imputation
print((pd.isnull(df)).sum())
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors = 100)
# df[features] = imputer.fit_transform(df[features])
# print((pd.isnull(df)).sum())

df[features].hist(figsize = (60, 50))


# replace PREDICTED_Hc with difference of two days.
# values = []
# last_value = None
# for i, value in enumerate(df["PREDICTED_Hc"]):
#     if not math.isnan(value):
#         # print(value)
#         if last_value is not None:
#             values.append(value - last_value)
#         else:
#             values.append(0)
#         last_value = value
#     else:
#         values.append(value)
#         last_value = None 
# # print(values)
# df["diff_Hc"] = values
# features.append("diff_Hc")
# print("min:")
# print(df[features].min())

In [ ]:
original_x = []
original_y = []
for i, season in enumerate(seasons):
    _x = (df[features].loc[season, :]).to_numpy()
    # print(_x.shape)
    _x = np.concatenate((_x, np.zeros((season_max_length - len(season), len(features)))), axis = 0)
    # print(season)
    # print(_x.shape)
    
    add_array = np.zeros(season_max_length - len(season))
    add_array[:] = np.NaN
    # print(len(season),add_array[1:].shape)
    _y_same_day = df.loc[season,:][['LT50']].to_numpy() - df.loc[season,:][['PREDICTED_Hc']].to_numpy()
    _y_same_day = np.concatenate((_y_same_day.flatten(), add_array), axis = 0)[:, None]
    _y_next_day = np.concatenate((np.array([np.NaN]), _y_same_day.flatten()[:-1]), axis = 0)[:, None]
    # print(_y_same_day.shape, _y_next_day.shape)
    _y = np.concatenate((_y_same_day, _y_next_day), axis = 1)
          
    # df_s = df.loc[season,:]
    # _y = np.zeros(season_max_length)
    # _y[_y == 0] = np.NaN
    # season_not_null_idx = df_s[df_s['LT50'].notnull()].index
    # _y[(season_not_null_idx - season[0]).to_numpy()] = df.loc[season_not_null_idx, :]['LT50'].to_numpy()# - df.loc[season_not_null_idx, :]['PREDICTED_Hc'].to_numpy()
    # # print(y_gt)
    # # _y = ([(season_not_null_idx - season[0]).tolist(), y_gt.tolist()])
    # # print(_x)
    # print(len(_y), _y)
    # input()
    original_x.append(_x)
    original_y.append(_y)
original_x = np.array(original_x)
original_y = np.array(original_y)
# data pre-processed
print(np.array(original_x).shape)
print(np.array(original_y).shape)

# print(np.isnan(original_x).sum(axis = 1))

    
x_all = data_prepocess(original_x, is_one_hot = False, is_normal = True)
y_all = np.array(original_y)
print(x_all.shape)
print(y_all.shape)

### Network Architecture

In [ ]:
class net(nn.Module):
    def __init__(self, input_size):
        super(net, self).__init__()
        self.num_out = len(label)
        self.numLayers = 1
        self.memory_size = 1024

        self.linear1 = nn.Linear(input_size, 256)
        self.linear_extra = nn.Linear(256, 512)
        self.rnn = nn.GRU(input_size=512, hidden_size=self.memory_size, num_layers=self.numLayers, batch_first=True)
        self.dropout = nn.Dropout(p=0.7)
        self.linear_s_1 = nn.Linear(self.memory_size, 512)
        self.linear_s_2 = nn.Linear(512, self.num_out)
        
        self.linear_n_1 = nn.Linear(self.memory_size, 512)
        self.linear_n_2 = nn.Linear(512, self.num_out)
        
    def forward(self, x, h=None):
        batch_dim, time_dim, state_dim = x.shape
        # print(batch_dim, time_dim, state_dim)
        out = self.linear1(x).relu()
        out = self.dropout(out)
        out = self.linear_extra(out).relu()
            
        if h is None:
            h = torch.zeros(self.numLayers, batch_dim, self.memory_size, device=x.device)
            # c = torch.zeros(self.numLayers, batch_dim, 1024, device=x.device)
        # print(out.shape, h.shape)
        # out, h_next = self.rnn(out, (h, c))
        out, h_next = self.rnn(out, h)
        # out = out[:, -1, :].relu()
        
        out_s = self.dropout(out)
        out_s = self.linear_s_1(out_s).relu()
        out_s_last = out_s.detach().clone()
        out_s = self.dropout(out_s)
        out_s = self.linear_s_2(out_s)

        # out_n = self.dropout(out)
        # out_n = self.linear_s_1(out_n).relu()
        # out_n = self.dropout(out_n)
        # out_n = self.linear_s_2(out_n)
        # return out_s, out_n, h_next
        return out_s, out.detach(), out_s_last.detach()


### Set up training parameters


In [ ]:
numberOfEpochs = 1#500
batchSize = 12
random_seed = 0
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.manual_seed(random_seed)
save_name = "best_residual_rnn_gps.pt"
model = net(np.array(x_all).shape[-1])
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 0.08)
criterion = nn.MSELoss()
criterion.to(device)
log_dir = "./logs/"
log_train = "frost-mitigation/train_sday_gps"
log_val = "frost-mitigation/val_sday_whole_gps"
writer = SummaryWriter(log_dir)

### Set up torch dataset input

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
indices = np.arange(x_all.shape[0])
(
    x_train,
    x_test,
    y_train,
    y_test,
    indices_train,
    indices_test,
) = train_test_split(x_all, y_all, indices, test_size=0.07, shuffle = False)#, random_state = random_seed)
print(indices_train, indices_test)
print(x_all.shape)
print(y_all.shape)
print(x_train.shape)
print(y_train.shape)
# print(y_train[0])
print(y_test.shape)
# Be careful of replacing nan with zeros
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

train_dataset = TensorDataset(x_train, y_train)
trainLoader = DataLoader(train_dataset, batch_size=batchSize)

val_dataset = TensorDataset(x_test, y_test)
valLoader = DataLoader(val_dataset, batch_size=batchSize)

In [ ]:
best_model = None
min_test_loss = float("inf")
def get_valuable_idx(y):
    return np.argwhere(np.isnan(y) == False) 
def save_model(model, name):
    torch.save(model.state_dict(), './saved_model/{}'.format(name))
def load_model(model, name):
    model.load_state_dict(torch.load('./saved_model/{}'.format(name)))
    return model

### Training Loop

In [ ]:
from copy import deepcopy
np.set_printoptions(threshold=10000)
for epoch in range(numberOfEpochs):
  
  # Training Loop
    with tqdm(trainLoader, unit="batch") as tepoch: 
        model.train()

        tepoch.set_description(f"Epoch {epoch + 1}/{numberOfEpochs} [T]")
        total_loss = 0
        count = 0
        for i, (x, y) in enumerate(trainLoader):
            count += 1
            x_torch = x.to(device)
            y_torch = y.to(device)
            output_s, _, _ = model(x_torch) 
            # output_s, output_n, _ = model(x) 
            optimizer.zero_grad()       # zero the parameter gradients
            valuable_idx = get_valuable_idx(y[:, :, 0])
            # print(valuable_idx)
            # print(output_s.shape)
            # print(y.shape)
            loss_s = criterion(output_s[valuable_idx[0], valuable_idx[1], :], y_torch[:, :, 0][valuable_idx[0], valuable_idx[1]][:,None])
            # print(output_s[valuable_idx[0], valuable_idx[1]].view(-1).shape)
            # print(y[:, :, 0][valuable_idx[0], valuable_idx[1]].shape)           
            # print(output_s[valuable_idx[0], valuable_idx[1]].shape)
            # input()
            # valuable_idx = get_valuable_idx(y[:, :, 1])
            # loss_n = criterion(output_n[valuable_idx[0], valuable_idx[1]], y[:, :, 1][valuable_idx[0], valuable_idx[1]][:,None])
            
            loss = loss_s# + loss_n
            loss.backward()             # backward + 
            optimizer.step()            # optimize
            total_loss += loss.item()
            # tepoch.set_postfix(Train_Loss=loss.item())
            tepoch.set_postfix(Train_Loss=total_loss / count)
            tepoch.update(1)

        writer.add_scalar(log_train, total_loss / count, epoch)
        

  # Validation Loop
    with torch.no_grad():
        with tqdm(valLoader, unit="batch") as tepoch: 

            model.eval()

            tepoch.set_description(f"Epoch {epoch + 1}/{numberOfEpochs} [V]")
            total_loss = 0
            count = 0
            for i, (x, y) in enumerate(valLoader):
                count += 1
                x_torch = x.to(device)
                y_torch = y.to(device)
                pred_s, _, _ = model(x_torch)
                # output_s, output_n, _ = model(x) 
                valuable_idx = get_valuable_idx(y[:, :, 0])
                loss_s = criterion(pred_s[valuable_idx[0], valuable_idx[1]], y_torch[:, :, 0][valuable_idx[0], valuable_idx[1]][:,None])

                # valuable_idx = get_valuable_idx(y[:, :, 1])
                # loss_n = criterion(output_n[valuable_idx[0], valuable_idx[1]], y[:, :, 1][valuable_idx[0], valuable_idx[1]][:,None])

                # total_loss += (loss_s.item() + loss_n.item()) / 2
                total_loss += loss_s.item()
                tepoch.set_postfix(Val_Loss=total_loss / count)
                tepoch.update(1)
            avg_loss = total_loss / count
            writer.add_scalar(log_val, avg_loss, epoch)
            if min_test_loss > avg_loss:
                min_test_loss = avg_loss
                best_model = deepcopy(model)
                save_model(best_model, save_name)

### Data Preprocessing for GPs

In [ ]:
def get_um_set(x, y, ox, model, is_train = True):
    with torch.no_grad():
        PREDICTED_HC_IDX = 0
        model.eval()
        pred_s, pred_rnn, pred_last = model(x.to(device))
        pred_s, pred_rnn, pred_last = pred_s.cpu(), pred_rnn.cpu(), pred_last.cpu()
        y = y[:, :, 0]
        if is_train:
            valuable_idx = get_valuable_idx(y)
            gt, pred_s, pred_rnn, pred_last = y[valuable_idx[0], valuable_idx[1]], \
                                              pred_s[valuable_idx[0], valuable_idx[1]], \
                                              pred_rnn[valuable_idx[0], valuable_idx[1]], \
                                              pred_last[valuable_idx[0], valuable_idx[1]]
            predicted_hc = ox[valuable_idx[0], valuable_idx[1], PREDICTED_HC_IDX]
        else:
            valuable_idx = get_valuable_idx(y)
            gt, pred_s, pred_rnn, pred_last = y.view(-1), \
                                              pred_s.reshape(pred_s.size(0) * pred_s.size(1), pred_s.size(2)), \
                                              pred_rnn.reshape(pred_rnn.size(0) * pred_rnn.size(1), pred_rnn.size(2)), \
                                              pred_last.reshape(pred_last.size(0) * pred_last.size(1), pred_last.size(2))
            predicted_hc = ox[:, :, PREDICTED_HC_IDX].flatten()
        LT50_value_pred = torch.tensor(predicted_hc[:, None]) + pred_s
        LT50_value_gt = torch.tensor(predicted_hc) + gt
        # print(LT50_value.shape, pred_rnn.shape, pred_last.shape)
        # pred_rnn = torch.cat((pred_rnn.double(), LT50_value), dim = 1)
        # pred_last = torch.cat((pred_last.double(), LT50_value), dim = 1)
        # predicted_hc
        # print(pred_s.shape, predicted_hc.shape, pred_rnn.shape, pred_last.shape)
        # print(pred_rnn.shape, pred_last.shape)
        
        return gt, pred_s, predicted_hc, pred_rnn, pred_last

PAC_THRESHOLD = 75
def pca_rdc(train_set, test_set, pac_thres = PAC_THRESHOLD, n_components = None):
    from sklearn.decomposition import PCA
    if n_components is None:
        pca = PCA()
        pca.fit(train_set)
        idx_reduction = pca.explained_variance_ratio_ >= np.percentile(pca.explained_variance_ratio_, pac_thres)
        n_components = idx_reduction.sum()
    
    pca = PCA(n_components = n_components)
    pca.fit(train_set)

    return pca.transform(train_set).astype(float), pca.transform(test_set).astype(float), n_components    

In [ ]:
train_LT50_gt, train_LT50_pred, train_furguson, train_x_um_rnn, train_x_um_last = get_um_set(x_train, y_train, original_x[indices_train], best_model)
print(train_LT50_gt.shape, train_LT50_pred.shape, train_furguson.shape, train_x_um_rnn.shape, train_x_um_last.shape)
test_LT50_gt, test_LT50_pred, test_furguson, test_x_um_rnn, test_x_um_last = get_um_set(x_test, y_test, original_x[indices_test], best_model, is_train = False)
print(test_LT50_gt.shape, test_LT50_pred.shape, test_furguson.shape, test_x_um_rnn.shape, test_x_um_last.shape, y_test.shape)\

# torch.Size([812, 1024]) torch.Size([812]) torch.Size([756, 1024]) torch.Size([3, 252]) torch.Size([756])
train_x_um_rnn_rdc, test_x_um_rnn_rdc, n_components = pca_rdc(train_x_um_rnn, test_x_um_rnn, n_components = 3)
print(train_x_um_rnn.shape, test_x_um_rnn.shape)
valuable_idx = get_valuable_idx(y_test[:, :, 0])
truth = y_test[:,:, 0][valuable_idx[0],valuable_idx[1]][:, None]

baseline = torch.tensor([train_LT50_gt.mean()] * truth.shape[0])
# print(baseline)
# print(truth)
print(criterion(baseline, truth[:,0]))


### GP definition

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

### GP Training function

In [ ]:
# initialize likelihood and model

def IDW_GPS(train_x, train_y, test_x, test_y, test_LT50_y, training_iter = 2000, lr=0.01, verbose = False):
    train_x, train_y, test_x, test_y, test_LT50_y = \
    torch.tensor(train_x), torch.tensor(train_y), torch.tensor(test_x), torch.tensor(test_y), torch.tensor(test_LT50_y)
    train_x_gpu, train_y_gpu, test_x_gpu, test_y_gpu = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)
    print(train_x.shape, train_y.shape, test_x.shape, test_y.shape, test_LT50_y.shape)
    print(type(train_x), type(train_y), type(test_x), type(test_y), type(test_LT50_y))
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)
    model.to(device)
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters
    valuable_idx = get_valuable_idx(test_y)
    # print(test_y)
    # print(valuable_idx)
    val_idx = valuable_idx[1][valuable_idx[0] == 0]
    # print(valuable_idx)
    for i in range(1, test_y.shape[0]):
        val_idx = torch.cat((val_idx, valuable_idx[1][valuable_idx[0] == i] + test_y.shape[1] * i))
    # print(val_idx)
    valid_y = test_y[valuable_idx[0], valuable_idx[1]]
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    mll.to(device)
    best_loss = float('inf')
    best_model = None
    best_pred = None
    for i in tqdm(range(training_iter)):
        model.train()
        likelihood.train()
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x_gpu)
        # Calc loss and backprop gradients
        # print(train_y.shape)
        loss = -mll(output, train_y_gpu)
        loss.backward()
        
        model.eval()
        likelihood.eval()
        f_preds = model(test_x_gpu)
        y_preds = likelihood(model(test_x_gpu))

        f_mean = f_preds.mean
        f_var = f_preds.variance
        f_covar = f_preds.covariance_matrix

        y_mean = y_preds.mean
        y_var = y_preds.variance
        y_covar = y_preds.covariance_matrix

        # print(f_mean[val_idx].shape, valid_y.shape)
        val_loss = criterion(y_mean[val_idx], valid_y.to(device))
        # print(y_mean[val_idx].shape, valid_y.shape)
        if verbose and i % 100 == 0:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f   Val Loss: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item(),
                val_loss,
            ))
        if best_loss >= val_loss:
            best_model = model
            best_loss = val_loss
            best_pred = y_preds
        optimizer.step()
    return best_pred, val_idx, best_loss, best_model

In [ ]:
gp_best_loss, val_last_idx, gp_best_pred, gp_best_model = IDW_GPS(train_x_um_rnn, train_LT50_gt, test_x_um_rnn, y_test[:, :, 0], test_LT50_gt, training_iter = 1, verbose = True)

### Plotting functions

In [ ]:
def _graph_bar(graph_name, prediction, ground_truth):
    print("p", prediction[:5])
    print("gt", ground_truth[:5])

    plt.figure(figsize=(16,9))

    ind = np.arange(prediction.shape[0])

    width = 0.3

    plt.bar(ind, prediction , width, label='Prediction')
    plt.bar(ind + width, ground_truth, width, label='GT')

    plt.xlabel('Example Index')
    plt.ylabel('Temperature. (C)')
    plt.title(graph_name)

    plt.legend(loc='best', prop={'size': 20})
    plt.savefig('bargraph.png')
    #plt.show()
    return

def _graph_bar_diff(graph_name, prediction, ground_truth):
    prediction = np.absolute(prediction)
    ground_truth = np.absolute(ground_truth)

    diff = ground_truth - prediction

    plt.figure(figsize=(16,9))

    ind = np.arange(prediction.shape[0])

    width = 0.3

    plt.bar(ind, diff, width, label='GT - Pred')

    plt.xlabel('Example Index')
    plt.ylabel('Temperature. (C)')
    plt.title(graph_name)

    plt.legend(loc='best', prop={'size': 20})
    plt.savefig('diffbargraph.png')
    #plt.show()
    return

def _graph_scatter(graph_name, ground_truth, prediction):
    prediction = np.absolute(prediction)
    print(np.amin(prediction), np.amax(prediction))

    ground_truth = np.absolute(ground_truth)
    print(np.amin(ground_truth), np.amax(ground_truth))

    plt.figure(figsize=(16,9))

    plt.title(graph_name)

    plt.scatter(x=ground_truth, y=prediction)

    default_x_ticks = range(ground_truth.shape[0])

    plt.xticks(default_x_ticks, ground_truth)

    plt.xlabel('ground_truth (C)')

    plt.ylabel('prediction (C)')

    plt.axis('square')
    plt.savefig('scatter.png')
    return

def _graph_line(graph_name, LT_values):
    x_axis = list(range(0, len(LT_values)))

    plt.figure(figsize=(16,9))

    plt.title(graph_name)
    
    for ltv in LT_values:
        if len(ltv) > 3:
            LT_v, x, label, lower, upper, c_lable = ltv
            plt.plot(x, LT_v, label = label)
            plt.fill_between(x, lower, upper, alpha=0.5, label = c_lable)
        else:
            LT_v, x, label = ltv
            plt.plot(x, LT_v, label = label)
                  
    plt.xlabel('days')

    plt.ylabel('LT50 values')
    plt.legend(prop={'size': 20})
    plt.savefig('linegraph.png')
    return

def confidence_region_95(pred_model):
    std2 = pred_model.stddev.mul_(1.96)
    mean = pred_model.mean
    return mean.sub(std2), mean.add(std2)

def run_and_plot(test_pred, val_idx, test_LT50_gt):
    test_LT50_gt = test_LT50_gt.to(device)
    output_mean = test_pred.mean.cpu().numpy()
    # lower, upper = test_pred.confidence_region()
    lower, upper = confidence_region_95(test_pred)
    truth = test_LT50_gt[val_idx].cpu().numpy()[:, None]
    print(output_mean.shape, test_furguson.shape, lower.shape, upper.shape)
    print("mean loss:", criterion(test_pred.mean[val_idx], test_LT50_gt[val_idx]))
    print("upper loss:", criterion(upper[val_idx], test_LT50_gt[val_idx]))
    in_range = (test_LT50_gt[val_idx] < upper[val_idx]) * (test_LT50_gt[val_idx] > lower[val_idx])
    print(in_range)
    print("ratio of gt in confidence range: ", in_range.sum() / len(in_range))
    print("ratio of under gt: ", (test_LT50_gt[val_idx] < upper[val_idx]).sum() / len(in_range))
    # print(truth.shape)
    LT50_prediction = output_mean[val_idx]
    LT50_truth = truth[:, 0]
    lower = lower.cpu()
    upper = upper.cpu()
    lower += test_furguson
    upper += test_furguson
    _graph_bar("Prediction vs Ground Truth (Difference bwt Hc and LT50 GT) Same day", LT50_prediction, LT50_truth, )
    _graph_scatter("Prediction vs Ground Truth (Difference bwt Hc and LT50 GT) Same day", LT50_prediction, LT50_truth, )
    _graph_bar_diff("GT - Prediction (Same Day)", LT50_truth, LT50_prediction)
    # print(test_furguson[val_last_idx].shape, test_pred.shape, truth.shape)
    _graph_line("LT50 lines (Residual)", [(test_furguson, np.arange(test_furguson.shape[0]), "Furguson(with linear interp)"), \
                               (output_mean + test_furguson, np.arange(output_mean.shape[0]), "Pred_GPs_Mean", lower.numpy(), upper.numpy(), "Confidence Region(95% Interval)"), \
                               (truth.flatten() + test_furguson[val_last_idx], val_last_idx, "Ground Truth")])

In [ ]:
with torch.no_grad():
    run_and_plot(gp_best_loss, val_last_idx, test_LT50_gt)

### Legacy code

In [ ]:
valuable_idx = get_valuable_idx(y_test[:,:, 0])
with torch.no_grad():
    output_s, output_n, _= best_model(x_test.to(device))
output = output_s[valuable_idx[0],valuable_idx[1]].cpu().numpy()
truth = y_test[:,:, 0][valuable_idx[0],valuable_idx[1]].numpy()[:, None]
# print(output.shape, truth.shape)
print(criterion(output_s[valuable_idx[0],valuable_idx[1]], y_test[:,:, 0][valuable_idx[0],valuable_idx[1]][:, None].to(device)))
print(output.shape)
print(truth.shape)
LT50_prediction = output[:, 0]
LT50_truth = truth[:, 0]
print(LT50_truth)
print(LT50_prediction)
_graph_bar("Prediction vs Ground Truth (Difference bwt Hc and LT50 GT) Same day", LT50_truth, LT50_prediction)
_graph_scatter("Prediction vs Ground Truth (Difference bwt Hc and LT50 GT) Same day", LT50_truth, LT50_prediction)
_graph_bar_diff("GT - Prediction (Next Day)", LT50_truth, LT50_prediction)
print(output.shape)
_graph_line("LT50 lines", [(test_furguson, np.arange(test_furguson.shape[0]), "pred_hc(with linear interp)"), \
                           (output.flatten() + test_furguson[val_last_idx], val_last_idx, "pred_LT_rnn"), \
                           (truth.flatten() + test_furguson[val_last_idx], val_last_idx, "ground truth")])